# THỐNG KÊ DỮ LIỆU

1. Yêu cầu chung: Thống kê được số lượng ảnh trong từng phân loại mà mỗi SV đóng góp

2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/1Uj0V9URNHpzSHeXHSB89AoGCjGki8Yra
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước https://colab.research.google.com/drive/1bUmXMM_ggnEXKo2qylfe6h0JnQuos-8_
- Output:
    + File CarDataset-1.csv - Thống kê số lượng ảnh theo từng SV
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: MSSV, All, Số lượng.
        
        Ví dụ: 20221234, All, 1.2
      + Số lượng có thể là số thực vì một ảnh có thể có đóng góp từ nhiều sinh viên
    + File CarDataset-2.csv - Thống kê số lượng ảnh theo từng Hiệu xe mà từng SV đóng góp
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: MSSV, Hiệu xe, Số lượng.
        
        Ví dụ: 20221234, Honda, 1.2
      + Số lượng có thể là số thực vì một ảnh có thể có đóng góp từ nhiều sinh viên
      + Chỉ chấp nhận file .jpg, .jpeg và .png
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: 17:00 - ??/??/2024

4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.


In [24]:
rm -rf "/kaggle/working/CarDataset-1.csv" "/kaggle/working/CarDataset-2.csv" "/kaggle/working/InvalidNames.csv"

In [25]:
import os
import re
import csv
from collections import defaultdict

# Danh sách các hãng xe với CategoryID
category_mapping = {
    0: 'Others',
    1: 'Honda',
    2: 'Hyundai',
    3: 'KIA',
    4: 'Mazda',
    5: 'Mitsubishi',
    6: 'Suzuki',
    7: 'Toyota',
    8: 'VinFast'
}

# Tạo mapping ngược từ tên hãng sang CategoryID
brand_to_id = {v: k for k, v in category_mapping.items()}

In [26]:
# Hàm kiểm tra định dạng tên file
def is_valid_filename(filename, brand):
    pattern = f"^(\d+(-\d+)*?)\\.{brand}\\.\d+\\.(jpg|jpeg|png)$"
    return re.match(pattern, filename, re.IGNORECASE)

# Hàm tính tỷ lệ đóng góp của mỗi MSSV
def parse_mssv(filename):
    mssv_part = filename.split('.')[0]
    mssvs = mssv_part.split('-')
    return {mssv: 1 for mssv in mssvs}  # Mỗi MSSV nhận toàn bộ số lượng ảnh

In [27]:
# Thống kê số lượng ảnh theo MSSV
all_counts = defaultdict(int)
brand_counts = defaultdict(lambda: defaultdict(int))
invalid_files = []  # Lưu danh sách các file sai định dạng

def process_images(root_dir):
    for brand in category_mapping.values():
        brand_dir = os.path.join(root_dir, brand)
        if not os.path.isdir(brand_dir):
            continue
        for file in os.listdir(brand_dir):
            file_path = os.path.join(brand, file)
            if is_valid_filename(file, brand):
                mssv_contributions = parse_mssv(file)
                for mssv, contribution in mssv_contributions.items():
                    all_counts[mssv] += contribution
                    brand_counts[mssv][brand] += contribution
            else:
                invalid_files.append((file_path, brand_to_id[brand]))

# Ghi file CarDataset-1.csv (sắp xếp theo số lượng giảm dần)
def write_dataset1(output_file):
    sorted_counts = sorted(all_counts.items(), key=lambda x: x[1], reverse=True)
    with open(output_file, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['MSSV', 'All', 'Số lượng'])
        for mssv, count in sorted_counts:
            writer.writerow([mssv, 'All', round(count, 2)])

# Ghi file CarDataset-2.csv (sắp xếp theo hiệu xe, sau đó là số lượng giảm dần)
def write_dataset2(output_file):
    sorted_brand_counts = []
    for mssv, brands in brand_counts.items():
        for brand, count in brands.items():
            sorted_brand_counts.append((mssv, brand, count))
    sorted_brand_counts.sort(key=lambda x: (x[1], -x[2]))  # Sắp xếp theo hiệu xe, sau đó là số lượng giảm dần
    with open(output_file, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['MSSV', 'Hiệu xe', 'Số lượng'])
        for mssv, brand, count in sorted_brand_counts:
            writer.writerow([mssv, brand, round(count, 2)])

# Ghi file chứa các file sai định dạng
def write_invalid_files(output_file):
    with open(output_file, mode='w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ImageFullPath', 'CategoryID'])
        for file_path, category_id in invalid_files:
            writer.writerow([file_path, category_id])

In [28]:
root_directory = "/kaggle/input/car-cs114/dataset"  # Thay bằng đường dẫn thư mục cha
process_images(root_directory)
write_dataset1("CarDataset-1.csv")
write_dataset2("CarDataset-2.csv")
write_invalid_files("InvalidNames.csv")
print("Đã tạo xong CarDataset-1.csv, CarDataset-2.csv và InvalidNames.csv")

Đã tạo xong CarDataset-1.csv, CarDataset-2.csv và InvalidNames.csv


In [29]:
import pandas as pd

path = "/kaggle/working/CarDataset-1.csv"
df = pd.read_csv(path)
df

,MSSV,All,Số lượng
0,22521027,All,11969
1,22520195,All,11969
2,22521060,All,11969
3,22520459,All,3779
4,22520507,All,3779
5,22520862,All,3779
6,22520394,All,3367
7,22520395,All,3367
8,22520779,All,2721
9,22521587,All,2721


In [30]:
path_2 = "/kaggle/working/CarDataset-2.csv"
df2 = pd.read_csv(path_2)
df2

,MSSV,Hiệu xe,Số lượng
0,22521027,Honda,1000
1,22520195,Honda,1000
2,22521060,Honda,1000
3,22520394,Honda,387
4,22520395,Honda,387
...,...,...,...
416,22520223,VinFast,20
417,22520213,VinFast,20
418,22521463,VinFast,15
419,22521213,VinFast,15


In [32]:
path_3 = "/kaggle/working/InvalidNames.csv"
df3 = pd.read_csv(path_3)
df3

,ImageFullPath,CategoryID
0,Others/22520348-22520530-22520837.MG.1.jpg,0
1,Others/22520223-22520213.Nissan.8.jpg,0
2,Others/21522373-21522499.LandRover.16.jpg,0
3,Others/21522373-21522499.Nissan.7.jpg,0
4,Others/22520223-22520213.Peugeot.6.jpg,0
...,...,...
1302,VinFast/21522500-21522771.VinFast20.jpg,8
1303,VinFast/21520930-21522924.VinFast.56-.png,8
1304,VinFast/21522500-21522771.VinFast25.jpg,8
1305,VinFast/21522500-21522771.VinFast18.jpg,8


In [33]:
!zip -r "stat.zip" "/kaggle/working"

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/InvalidNames.csv (deflated 92%)
  adding: kaggle/working/CarDataset-2.csv (deflated 79%)
  adding: kaggle/working/CarDataset-1.csv (deflated 69%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
